In [ ]:
!cd /content && rm -rf grover && git clone https://github.com/rowanz/grover.git
%cd /content/grover
!pip install -r requirements-gpu.txt
!python download_model.py large

Cloning into 'grover'...
remote: Enumerating objects: 104, done.
remote: Total 104 (delta 0), reused 0 (delta 0), pack-reused 104
Receiving objects: 100% (104/104), 672.23 KiB | 2.74 MiB/s, done.
Resolving deltas: 100% (43/43), done.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-15b8905d83af>", line 2, in <module>
    get_ipython().magic('cd /content/grover')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, 

FileNotFoundError: ignored

In [ ]:
import json
from datetime import datetime
from textwrap import wrap

In [ ]:
!pip install numpy --upgrade
!pip install tensorflow-gpu==1.15.0
!pip install tensorboard==2.8.0
from tensorflow.python.util.tf_export import keras_export

In [ ]:
pip install tika

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32893 sha256=d45e1dedf8ff905cbefd2e9a189fa8de601e8785256c0a9bed76aa5efbc5547e
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


In [ ]:
#tika funtions for text extraction
from tika import parser as p
import pandas as pd
import requests
    
#function pdf -> text extraction
def get_data_from_web(url):
    response = requests.get(url)
    results = p.from_buffer(response.content)
    return results

def get_data_from_given_path(file_path):
    results = p.from_file(file_path)
    return results
def extract_text(file_path):
    final_text = []
    #extract text from any website pdf
    #pdf_url = "https://www.bl.uk/learning/resources/pdf/makeanimpact/sw-transcripts.pdf"
    #results = get_data_from_web(pdf_url)
    #print("\nFile Content: \n{}".format(results["content"].strip()))
    pdf_file_path = file_path
    results = get_data_from_given_path(pdf_file_path)
    text = results["content"].strip()
    #final_text = [string for string in text if string != ""]
    final_text = results['metadata']
    return text
    #print(results["content"].strip())

In [ ]:
df = pd.read_csv('/content/Final_bik_dataset.tsv', sep='\t',encoding = "ISO-8859-1", nrows=214)

In [ ]:
#convert authors to a list of strings
def author_list(authors):
  x=authors.split(", ")
  return list(x)

In [ ]:
authors=df['Authors'].tolist()
new_authors=[]


for element in authors:
  new_element=author_list(element)
  new_authors.append(new_element)

#print(new_authors)

df['new_authors']=new_authors

In [ ]:
#convert date into grover format
year=df['Year'].tolist()
month=df['Month'].tolist()
def month_conversion(month):
  try:
    new_month=month.split("-")
    new_month_num=new_month[0]
    new_month_num=int(new_month_num)
    new_month='0'+str(new_month_num)
  except:
    new_month='06'
  return new_month

new_month_list=[]
for index in range(len(month)):
  new_month_list.append(month_conversion(month[index]))

In [ ]:
dates_list=[]
for index in range(len(year)):
  date=str(year[index])+'-'+str(new_month_list[index])+'-01'
  dates_list.append(date)

In [ ]:
#get downloaded pdfs from Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
def separate_by_dash(name):
  new_name=name.split(".")
  return new_name[0]

In [ ]:
#remane all files in pdfs folder
folder="/content/drive/MyDrive/DSCI 550/pdfs"

for count, filename in enumerate(os.listdir(folder)):
    dst = separate_by_dash(filename)+".pdf"
    src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
    dst =f"{folder}/{dst}"

    os.rename(src, dst)

In [ ]:
#make a list of input jsons to feed into grover
titles=df['Title'].tolist()

input_json_list=[]

for index in range(len(titles)):
  publish_date=dates_list[index]
  pdf_index=index+2
  new_pdf_name=str(pdf_index)
  #print(new_pdf_name)
  try: #iff paper exists and is the name of the doi
    input_json={}
    input_json['title']=titles[index]
    input_json['domain']='https://libraries.usc.edu/'
    input_json['text']=extract_text('/content/drive/MyDrive/DSCI 550/pdfs/'+new_pdf_name+'pdf.pdf') #deffinitely use try/except here
    input_json['summary']=''
    input_json['authors']=new_authors[index]
    publish_date=dates_list[index]
    grover_publish_date = datetime.strptime(publish_date, "%Y-%m-%d").strftime("%m-%d-%Y")
    input_json['publish_date']=grover_publish_date

    input_json_list.append(input_json)
  except:
    pass

In [ ]:
#make a list of fake texts
fake_text_final=[]

In [ ]:
#loop through list of jsons to generate fake texts
fake_texts=[]
WRAP_EVERY_N_CHARACTERS = 100
for element in input_json_list:
  print(1)
  input_json=element
  
  #grover
  with open('./input.json', 'w') as f:
    f.write(json.dumps(input_json))

  !PYTHONPATH=$(pwd) python sample/contextual_generate.py -model_config_fn lm/configs/large.json \
    -model_ckpt models/large/model.ckpt \
    -metadata_fn input.json \
    -out_fn output.json
  
  with open('./output.json') as f:
    file_contents = f.read()
    data = json.loads(file_contents)

  generated_article = data['gens_article'][0]
  wrapped_article = '\n'.join(wrap(generated_article, WRAP_EVERY_N_CHARACTERS))
  #print(wrapped_article)
  fake_texts.append(wrapped_article)


In [ ]:
#put 121 fake texts into a dataframe and convert into txt file (note to do this 5 times to get 500 texts)
df_text=pd.DataFrame()

df_text['texts']=fake_texts

In [ ]:
df_text.to_csv('/content/pandas_t.txt', sep='\t', mode='a')

In [ ]:
#now compile 500 fake texts into one txt file
df1=pd.read_csv('/content/pandas_t.txt',sep='\t',nrows=214)
df2=pd.read_csv('/content/pandas_t2.txt',sep='\t',nrows=214)
df3=pd.read_csv('/content/pandas_t3.txt',sep='\t',nrows=214)
df4=pd.read_csv('/content/pandas_t4.txt',sep='\t',nrows=214)
df5=pd.read_csv('/content/pandas_t5.txt',sep='\t',nrows=214)

In [ ]:
first=df1['texts'].tolist()
second=df2['texts2'].tolist()
third=df3['texts3'].tolist()
fourth=df4['texts4'].tolist()
fifth=df5['texts5'].tolist()
fifth=fifth[:16]

In [ ]:
first.extend(second)
first.extend(third)
first.extend(fourth)
first.extend(fifth)

In [ ]:
df_texts=pd.DataFrame()

df_texts['texts']=first

In [ ]:
df_texts.to_csv('/content/500_texts.txt', sep='\t', mode='a')

NameError: ignored